# GEOtiled: A Scalable Workflow for Generating Large Datasets of High-Resolution Terrain Parameters

Terrain parameters such as slope, aspect and hillshading can be computed from a Digital Elevation Model (DEM) which is a representation of elevation data of the surface of the earth. In SOMOSPIE these terrrain parameters are used to downscale satellite-derived soil moisture data.

GEOtiled comprises three stages: (i) the partition of the DEM into tiles, each with a buffer region; (ii) the computation of the terrain parameters for each individual tile; and finally, (iii) the generation of a mosaic for each parameter from the tiles by averaging the values of the pixels that overlap between the tiles (i.e., pixels within the buffer regions). 

<p align="center">
<img src="../../../somospie_pngs/geotiled.png" width="500"/>
</p>

<p align="center">
<b>Figure 1: </b>GEOtiled workflow.
</p>

This notebook uses DEMs from [USGS 3DEP products](https://www.usgs.gov/3d-elevation-program/about-3dep-products-services) to compute 3 topographic parameters: Aspect, Hillshading and Slope.

Before running the workflow on this notebook, go to [USGS Data Download Application](https://apps.nationalmap.gov/downloader/#/elevation) and use the map to look for available DEM data. Once you have selected a specific region and resolution, you can get a txt file with all the individual download links for the tiles corresponding to your selection. This txt file will serve as input to this notebook which uses the links to download the tiles and compute the parameters.

The terrain parameters are by default generated as GeoTIFF files, the option to change their format and stack them if needed is included at the end.

## Environment setup
Run the following code boxes to set the working directories and packages necessary for this workflow.

In [1]:
from tools import *

In the code cell bellow specify the inputs to the workflow:
* **in_file:** path to the txt file with download links for DEM tiles you wish to use.
* **out_folder:** path to the folder you want the terrain parameters to be stored.
* **projection:** The projection can be an identifier such as 'EPSG:3572' or the path to a wkt file. To compute terrain parameters correctly, the DEM must be in a projection whose x, y and z coordinates are expressed in the same units, Albers Equal Area USGS projection was used for CONUS, but you can modify it depending on the region you are analyzing.
* **n_tiles:** Number of tiles you want the workflow to partition the DEM into.

In [2]:
in_file = './data.txt'
out_folder = '/media/volume/sdb'
projection = 'albers_conus_reference.wkt'
n_tiles = 16

## Pre-processing of the DEM
Downloads each tile from the URLs listed in input txt file and store them in the specified output folder on a subdirectory named tiles.

In [ ]:
tiles_folder = os.path.join(out_folder, 'tiles')
Path(out_folder).mkdir(parents=True, exist_ok=True)
Path(tiles_folder).mkdir(parents=True, exist_ok=True)

print('Downloading tiles...')
download_dem(in_file, tiles_folder)
print('Download completed.')

Merges downloaded tiles into a single raster (mosaic).

In [ ]:
raster_list = glob.glob(tiles_folder + '/*')
mosaic_path = os.path.join(out_folder, 'mosaic.tif')

merge_tiles(raster_list, mosaic_path)

# Optional: delete all tiles after building mosaic
shutil.rmtree(tiles_folder)

Reprojects the mosaic to ensure coordinates and elevation values are in meters.

In [ ]:
dem_path = os.path.join(out_folder, 'elevation.tif')
reproject(mosaic_path, dem_path, projection)

# Optional: delete mosaic with initial projection
os.remove(mosaic_path)

## 1. Crop DEM into tiles
Crops the DEM into the number of tiles you selected and adds a buffer region to each of these tiles to prevent boundary artifacts. This phenomenon happens because computation at a single pixel uses values from adjacent pixels, therefore when there are no buffers the accuracy of the computation process is impacted. 

In [3]:
elevation_tiles = os.path.join(out_folder, 'elevation_tiles')
Path(elevation_tiles).mkdir(parents=True, exist_ok=True)

crop_into_tiles(dem_path, elevation_tiles, n_tiles)

0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.


## 2. Compute terrain parameters for each tile
Computes the terrain parameters (Aspect, Hillshading and Slope) in parallel per tile using multiprocessing.

In [4]:
aspect_tiles = os.path.join(out_folder, 'aspect_tiles')
hillshading_tiles = os.path.join(out_folder, 'hillshading_tiles')
slope_tiles = os.path.join(out_folder, 'slope_tiles')

Path(aspect_tiles).mkdir(parents=True, exist_ok=True)
Path(hillshading_tiles).mkdir(parents=True, exist_ok=True)
Path(slope_tiles).mkdir(parents=True, exist_ok=True)

pool = multiprocessing.Pool() 
pool.map(compute_geotiled, sorted(glob.glob(elevation_tiles + '/*.tif')));

## 3. Build mosaic from the tiles
Cleans the repetitive information from the buffer region by building a mosaic with the average values of the overlapping regions within the tiles.

In [6]:
build_mosaic(sorted(glob.glob(aspect_tiles + '/*.tif')), os.path.join(out_folder, 'aspect.tif'))
build_mosaic(sorted(glob.glob(hillshading_tiles + '/*.tif')), os.path.join(out_folder,'hillshading.tif'))
build_mosaic(sorted(glob.glob(slope_tiles + '/*.tif')), os.path.join(out_folder, 'slope.tif'))

Input file size is 4900, 3110
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 4900, 3110
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 4900, 3110
0...10...20...30...40...50...60...70...80...90...100 - done.


## Change raster format and projection (optional)
Terrain parameters will be stored by default in GeoTIFF format, run the following cells if you wish to get them in a different format.

Change the format and extension to the one you wish the files to be converted to in the next code cell. Go to [GDAL raster drivers](https://gdal.org/drivers/raster/index.html) to check which formats are available.

In [ ]:
raster_format = 'GTiff'
extension = '.tif'
projection = 'EPSG:4326'

In [ ]:
param_files = sorted(glob.glob(os.path.join(out_folder, '*.tif')))

for f in param_files:
    change_raster_format(f,  f[0:-4]+extension, raster_format)
    os.remove(f) # Optional: delete parameters with initial format

In [ ]:
param_files = sorted(glob.glob(os.path.join(out_folder, '*' + extension)))

for f in param_files:
    reproject(f, f, projection)

## Create a stack with terrain parameters (optional)
If you want to get a stack of the terrain parameters run the following code box. The stack file will be stored in the ouput folder you specified at the start of this notebook.

In [ ]:
param_list = sorted(glob.glob(os.path.join(out_folder, '*' + extension)))
stack_file = os.path.join(out_folder ,'stack.tif')

build_stack(param_list, stack_file)